In [ ]:
'''
Effn-B4
1536 image size
8 epochs
3 Folds
'''

In [ ]:
!pip install tensorflow-addons

In [ ]:
!pip install -q efficientnet

In [ ]:
import os
import re
import seaborn as sns
import numpy as np
import pandas as pd
import math

from matplotlib import pyplot as plt

from sklearn import metrics
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow_addons as tfa
from sklearn.model_selection import StratifiedKFold

import efficientnet.tfkeras as efn

from kaggle_datasets import KaggleDatasets

In [ ]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# Data access
GCS_PATH = KaggleDatasets().get_gcs_path('panda-dataset-medium-36-256-256')

# Configuration
EPOCHS = 12
BATCH_SIZE = 4 * strategy.num_replicas_in_sync
img_size = 1536

In [ ]:
GCS_PATH

In [ ]:
train = pd.read_csv('../input/prostate-cancer-grade-assessment/train.csv')

In [ ]:
sns.countplot(train['isup_grade'])

In [ ]:
train_images_path = '../input/panda-dataset-medium-36-256-256/'

In [ ]:
img = plt.imread(train_images_path + train.loc[1]['image_id'] + '.png')
plt.imshow(img)
print(img.shape)

In [ ]:
train["image_path"] = train["image_id"].apply(lambda x: GCS_PATH + '/' + x + '.png')

In [ ]:
train

In [ ]:
n_splits = 3
skf = StratifiedKFold(n_splits = 3)

result = []   
for train_idx, val_idx in skf.split(train['image_path'], train["isup_grade"]):
    train_fold = train.iloc[train_idx]
    val_fold = train.iloc[val_idx]
    result.append((train_fold, val_fold))
    
train_fold_1, val_fold_1 = result[0][0],result[0][1]
train_fold_2, val_fold_2 = result[1][0],result[1][1]
train_fold_3, val_fold_3 = result[2][0],result[2][1]
# train_fold_4, val_fold_4 = result[3][0],result[3][1]
# train_fold_5, val_fold_5 = result[4][0],result[4][1]

In [ ]:
train_fold_1 = pd.read_csv('../input/pandakaranfoldsfinetuning/train_fold_1.csv')
val_fold_1   = pd.read_csv('../input/pandakaranfoldsfinetuning/val_fold_1.csv')

train_fold_2 = pd.read_csv('../input/pandakaranfoldsfinetuning/train_fold_2.csv')
val_fold_2   = pd.read_csv('../input/pandakaranfoldsfinetuning/val_fold_2.csv')

train_fold_3 = pd.read_csv('../input/pandakaranfoldsfinetuning/train_fold_3.csv') 
val_fold_3   = pd.read_csv('../input/pandakaranfoldsfinetuning/val_fold_3.csv')
# train_fold_4.to_csv('train_fold_4.csv',index=False)
# val_fold_4.to_csv('val_fold_4.csv',index=False)
# train_fold_5.to_csv('train_fold_5.csv',index=False)
# val_fold_5.to_csv('val_fold_5.csv',index=False)

In [ ]:
# xtrain, xval, ytrain, yval = train_test_split(train["image_path"], train["isup_grade"], test_size = 0.15, stratify = train["isup_grade"])

# df_train = pd.DataFrame({"image_path":xtrain, "isup_grade":ytrain})
# df_val = pd.DataFrame({"image_path":xval, "isup_grade":yval})

# df_train["isup_grade"] = df_train["isup_grade"].astype('int')
# df_val["isup_grade"] = df_val["isup_grade"].astype('int')

In [ ]:
# train_labels = pd.get_dummies(df_train['isup_grade']).astype('int32').values
# valid_labels = pd.get_dummies(df_val['isup_grade']).astype('int32').values

# print(train_labels.shape) 
# print(valid_labels.shape)

In [ ]:
def decode_image(filename, label=None, image_size=(img_size, img_size)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, size = image_size)
    
    if label is None:
        return image
    else:
        return image, label

def data_augment(image, label=None):
    image = tf.image.rot90(image)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if label is None:
        return image
    else:
        return image, label

def int_div_round_up(a, b):
    return (a + b - 1) // b

# NUM_TRAINING_IMAGES = df_train.shape[0]
# NUM_VALIDATION_IMAGES = df_val.shape[0]
# STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
# VALIDATION_STEPS = int_div_round_up(NUM_VALIDATION_IMAGES, BATCH_SIZE)
# print('Dataset: {} training images, {} validation images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES))

In [ ]:
# train_dataset = (
#     tf.data.Dataset
#     .from_tensor_slices((df_train['image_path'], train_labels))
#     .map(decode_image, num_parallel_calls=AUTO)
#     .map(data_augment, num_parallel_calls=AUTO)
#     .repeat()
#     .shuffle(512)
#     .batch(BATCH_SIZE)
#     .prefetch(AUTO))

In [ ]:
# valid_dataset = (
#     tf.data.Dataset
#     .from_tensor_slices((df_val['image_path'], valid_labels))
#     .map(decode_image, num_parallel_calls=AUTO)
#     .batch(BATCH_SIZE)
#     .prefetch(AUTO))

In [ ]:
LR_START = 0.0001
LR_MAX = 0.0001 * 1
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 0
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .7

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

rng = [i for i in range(EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
# # checkpoint
# filepath="effnB4-{epoch:02d}-{val_accuracy:.2f}.hdf5"
# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Model Definition

In [ ]:
def get_model():
    with strategy.scope():
        model = tf.keras.Sequential([
            efn.EfficientNetB4(
                input_shape=(img_size, img_size, 3),
                weights='imagenet',
                include_top=False
            ),
            L.GlobalAveragePooling2D(),
            L.Dense(1024, activation = 'relu',kernel_regularizer = tf.keras.regularizers.l2()), 
            L.Dropout(0.3),
            L.Dense(512, activation= 'relu'),
            L.Dense(6, activation='softmax')
        ])

    model.compile(
        optimizer='adam',
        loss = 'categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# FOLD 1

In [ ]:
train_paths_fold_1 = train_fold_1.image_path.values
val_paths_fold_1 = val_fold_1.image_path.values

train_labels_fold_1 = pd.get_dummies(train_fold_1['isup_grade']).astype('int32').values
val_labels_fold_1 = pd.get_dummies(val_fold_1['isup_grade']).astype('int32').values

train_dataset_fold_1 = (
    tf.data.Dataset
    .from_tensor_slices((train_paths_fold_1, train_labels_fold_1))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(data_augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO))

valid_dataset_fold_1 = (
    tf.data.Dataset
    .from_tensor_slices((val_paths_fold_1, val_labels_fold_1))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO))

NUM_TRAINING_IMAGES = train_fold_1.shape[0]
NUM_VALIDATION_IMAGES = val_fold_1.shape[0]
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
VALIDATION_STEPS = int_div_round_up(NUM_VALIDATION_IMAGES, BATCH_SIZE)
print('Dataset: {} training images, {} validation images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES))

In [ ]:
# checkpoint
filepath="eb4-finetune-fold1-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]


In [ ]:
model_fold_1 = get_model()
history1 = model_fold_1.fit(
    train_dataset_fold_1, 
    epochs = EPOCHS, 
    callbacks = [lr_callback, checkpoint],
    steps_per_epoch = STEPS_PER_EPOCH,
    validation_data = valid_dataset_fold_1,
    validation_steps = VALIDATION_STEPS
)

In [ ]:
acc = history1.history['accuracy']
val_acc = history1.history['val_accuracy']

loss = history1.history['loss']
val_loss = history1.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
#model_fold_1.save('effnB4_tpu_png_1536_fold_1.h5')

# Fold 2

In [ ]:
train_paths_fold_2 = train_fold_2.image_path.values
val_paths_fold_2 = val_fold_2.image_path.values

train_labels_fold_2 = pd.get_dummies(train_fold_2['isup_grade']).astype('int32').values
val_labels_fold_2 = pd.get_dummies(val_fold_2['isup_grade']).astype('int32').values

train_dataset_fold_2 = (
    tf.data.Dataset
    .from_tensor_slices((train_paths_fold_2, train_labels_fold_2))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(data_augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO))

valid_dataset_fold_2 = (
    tf.data.Dataset
    .from_tensor_slices((val_paths_fold_2, val_labels_fold_2))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO))

NUM_TRAINING_IMAGES = train_fold_2.shape[0]
NUM_VALIDATION_IMAGES = val_fold_2.shape[0]
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
VALIDATION_STEPS = int_div_round_up(NUM_VALIDATION_IMAGES, BATCH_SIZE)
print('Dataset: {} training images, {} validation images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES))

model_fold_2 = get_model()
history2 = model_fold_2.fit(
    train_dataset_fold_2, 
    epochs = EPOCHS, 
    callbacks = [lr_callback],
    steps_per_epoch = STEPS_PER_EPOCH,
    validation_data = valid_dataset_fold_2,
    validation_steps = VALIDATION_STEPS
)

In [ ]:
acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']

loss = history2.history['loss']
val_loss = history2.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model_fold_2.save('effnB4_tpu_png_1536_fold_2.h5')

# Fold 3

In [ ]:
train_paths_fold_3 = train_fold_3.image_path.values
val_paths_fold_3 = val_fold_3.image_path.values

train_labels_fold_3 = pd.get_dummies(train_fold_3['isup_grade']).astype('int32').values
val_labels_fold_3 = pd.get_dummies(val_fold_3['isup_grade']).astype('int32').values

train_dataset_fold_3 = (
    tf.data.Dataset
    .from_tensor_slices((train_paths_fold_3, train_labels_fold_3))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(data_augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO))

valid_dataset_fold_3 = (
    tf.data.Dataset
    .from_tensor_slices((val_paths_fold_3, val_labels_fold_3))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO))

NUM_TRAINING_IMAGES = train_fold_3.shape[0]
NUM_VALIDATION_IMAGES = val_fold_3.shape[0]
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
VALIDATION_STEPS = int_div_round_up(NUM_VALIDATION_IMAGES, BATCH_SIZE)
print('Dataset: {} training images, {} validation images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES))

model_fold_3 = get_model()
history3 = model_fold_3.fit(
    train_dataset_fold_3, 
    epochs = EPOCHS,
    callbacks = [lr_callback],
    steps_per_epoch = STEPS_PER_EPOCH,
    validation_data = valid_dataset_fold_3,
    validation_steps = VALIDATION_STEPS
)

In [ ]:
acc = history3.history['accuracy']
val_acc = history3.history['val_accuracy']

loss = history3.history['loss']
val_loss = history3.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model_fold_3.save('effnB4_tpu_png_1536_fold_3.h5')